## Machine Learning & Deep Learning: Cross-Validation 

Ein kleines Add-On, in dem ich Ihnen zeige, wie man die K-fold Cross-Validation anwendet

### Schritt 1: Importiere relevante Packages

Scikit-learn ist das bekanntest Python Package für Machine Learning. Es enthält die wichtigsten Modelle und Model Evaluation Metrics

In [ ]:
import pandas as pd 
import numpy as np
import helpers

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC

# Validation libraries
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, mean_squared_error, precision_score, precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score


# Display plots inside the notebook
%matplotlib inline
sns.set(style="white", palette="pastel")

# Ignore warning related to pandas_profiling
import warnings
warnings.filterwarnings('ignore') 

from pathlib import Path

# Display all dataframe columns in outputs (it has 63 columns, which is wider than the notebook)
# This sets it up to display with a horizontal scroll instead of hiding the middle columns
pd.set_option('display.max_columns', 100) 
pd.set_option('display.max_colwidth', -1)

## Schritt 2: Train und Test Datensatz festlegen

In [ ]:
# mit der Pickle Funktion packen wir den fertig bearbeiteten Dataframe in eine Datei, die wir nächste Woche einfach 
# importieren können. 
X = pd.read_pickle("X_df.pkl")
y = pd.read_pickle("y_df.pkl")

In [ ]:
seed = 5 # wenn wir den seed festlegen können wir den random-split später reproduzieren
test_size = 0.33 # d.h. 33% des Datensatzes werden zufällig ausgewählt und dem Test- Datensatz zugeordnet.

# Durch den Train Test Split teilen wir den Datensatz in einen Train-Datensatz und einen Test-Datensatz auf. 
# Das Modell wird auf dem Train-Datensatz trainiert und am Test-Datensatz getestet.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

#### Vorbereitung zur Evaluierung von Modellen:
1. An den gleichen Daten trainieren und testen: Wir wissen nicht, ob das Modell auch für andere Daten gute Vorhersagen trifft. 
2. **Train/Test Split**: Teilt den Datensatz in zwei Teile auf, sodass an unterschiedlichen Daten gelernt und getestet wird. Bessere Schätzung der Out-of-Sample-Performance (=Generalisierung) als 1.

### 2.1 K-fold Cross-Validation

Bei der K-fold Cross-Validation wir der Datensatz in K verschiedene Subsamples (=folds) unterteilt. Jeder fold wird dabei einmal als Test-Datensatz benutzt (während die restlichen **K-1** folds zum Training genutzt werden). So kann die Modell-Güte an verschiedenen Datensätzen getestet werden und ist somit objektiver. Das Ergebnis sind **K** Accuracy-Scores, aus denen i.d.R. der Mittelwert gebildet wird. 


<img src="img/cross_val.gif" alt="learning" style="width: 500px;"/>
Image Source: https://genome.tugraz.at/proclassify/help/pages/XV.html

In [ ]:
from sklearn.model_selection import cross_val_score

## Schritt 3: Modelle 

### 3.1. Logistische Regression

In [ ]:
# Modell anlegen. Der Hyperparameter C legt die Regularization Strength fest, d.h. wie sehr falsche Vorhersagen 
# "bestraft" werden. Große Werte von C erlauben Fehler und damit komplexere Modelle - dafür ist das Risiko von
# "Overfitting" erhöht. Kleine Werte von C bestrafen Fehler stärker, d.h. das Modell wird einfacher.
# Wenn es zu einfach wird droht "Underfitting"

log_model = LogisticRegression(C=1.0) # Lege ein Modell an. C ist hier auf dem Default. 
log_model.fit(X_train, y_train) # trainiere das Modell, sodass es bessere Vorhersagen trifft

In [ ]:
# Zunächst sagen wir unseren Testdatensatz mithilfe des Train Datensatzes vorher. 

y_pred_train = log_model.predict(X_train)

accuracy = accuracy_score(y_train, y_pred_train)
print("Train Accuracy: {}".format(accuracy))


# Nun sagen wir unseren Testdatensatz mithilfe des Test Datensatzes vorher. 
# Dieser wurde von unserem Modell noch nie gesehen. 

y_pred_test = log_model.predict(X_test)
    
log_accuracy = accuracy_score(y_test, y_pred_test)
print("\nTest Accuracy: {}".format(log_accuracy))

# Nun wenden wir die Cross-Validation mit 10 folds an
log_cv_accuracy = cross_val_score(log_model, X, y, cv=5, scoring='accuracy')
print("cross-validation scores", accuracy)

#### Cross-Validation Score

Nun wenden wir eine Cross-Validation mit 10 folds an. Hier nehmen wir nicht unseren aufgeteilten Datensatz (Xtrain) sondern den Gesamtdatensatz X. 

In [ ]:
# Nun wenden wir die Cross-Validation mit 10 folds an
log_cv_accuracy = cross_val_score(log_model, X, y, cv=10, scoring='accuracy')

print("Cross-validation scores:", log_cv_accuracy)
print("\nCross-validation score mean:", log_cv_accuracy.mean())

### 3. 2. Support Vector Classifier (SVC)

#### 3.2.1.1 Linear SVM

In [ ]:
# Legen Sie einen Linear Support Vector Classifier an
linear_svc = LinearSVC(C=0.001)

In [ ]:
# Trainiere das Modell mit unserem Train Datensatz
linear_svc = linear_svc.fit(X_train, y_train)

In [ ]:
# Zuerst sagen wir unsere Zielvariable mithilfe des Test Datensatzes vorher. 

y_pred_train = linear_svc.predict(X_train)

lin_svc_accuracy = accuracy_score(y_train, y_pred_train)
print("Train Accuracy: {}".format(lin_svc_accuracy))

# Nun sagen wir unsere Zielvariable mithilfe des Test Datensatzes vorher. 

y_pred_test = linear_svc.predict(X_test)
    
svm_lin_accuracy = accuracy_score(y_test, y_pred_test)
print("\nTest Accuracy: {}".format(svm_lin_accuracy))

#### Cross Validation Score

In [ ]:
# Nun wenden wir die Cross-Validation mit 10 folds an
linsvc_cv_accuracy = cross_val_score(linear_svc, X, y, cv=10, scoring='accuracy')

print("Cross-validation scores:", linsvc_cv_accuracy)
print("\nCross-validation score mean:", linsvc_cv_accuracy.mean())

#### 3.2.1.2 SVM mit RBF Kernel 

In [ ]:
# Legen Sie einen Support Vector Classifier mit RBF Kernel an
rbf_svc = SVC(kernel='rbf', random_state=0, gamma=0.03, C=1.0, probability=True)

# Trainieren Sie den Classifier
rbf_svc.fit(X_train, y_train)

In [ ]:
# Zuerst sagen wir unsere Zielvariable mithilfe des Test Datensatzes vorher. 

y_pred_train = rbf_svc.predict(X_train)

accuracy = accuracy_score(y_train, y_pred_train)
print("Train Accuracy: {}".format(accuracy))

# Nun sagen wir unsere Zielvariable mithilfe des Test Datensatzes vorher. 

y_pred_test = rbf_svc.predict(X_test)
    
rbf_svc_accuracy = accuracy_score(y_test, y_pred_test)
print("\nTest Accuracy: {}".format(rbf_svc_accuracy))

#### Cross Validation Score

In [ ]:
# Nun wenden wir die Cross-Validation mit 10 folds an
rbfsvc_cv_accuracy = cross_val_score(rbf_svc, X, y, cv=10, scoring='accuracy')

print("Cross-validation scores:", rbfsvc_cv_accuracy)
print("\nCross-validation score mean:", rbfsvc_cv_accuracy.mean())

### Compare all  CV Scores

In [ ]:
methodDict = {}
methodDict['Log_CV'] = log_cv_accuracy.mean()
methodDict['Linear_SVC_CV'] = linsvc_cv_accuracy.mean()
methodDict['RBF_Kernel_CV'] = rbfsvc_cv_accuracy.mean()

In [ ]:
def plotSuccess():
    s = pd.Series(methodDict)
    s = s.sort_values(ascending=False)
    plt.figure(figsize=(6,6))
    #Colors
    ax = s.plot(kind='bar') 
    for p in ax.patches:
        ax.annotate(str(round(p.get_height(),2)), (p.get_x() * 1.005, p.get_height() * 1.005))
    #plt.ylim([30.0, 90.0])
    plt.xlabel('Method')
    plt.ylabel('Percentage')
    plt.title('Success of methods')
     
    plt.show()

In [ ]:
#sns.set()

plotSuccess()

### Weitere Ressourcen:

- Chris Albon ist Autor von dem gern genutzten "Machine Learning with Python Cookbook". Auf seiner Seite finden sich ziemlich viele und gute Ressourcen https://chrisalbon.com/